In [1]:
'''
Created By Josh Stokley to import variables from a .mat database to python
The class provides-
variables:
 self.index ........................... index of wall corrosponding with index of database
 self.walldata ........................ large array that holds all variables to create tcl file
 self.wall_id ......................... unique id of wall
 
 
methods:
 def __init__(self, index) ... constructor: takes index of wall and imports variables from .mat database into an array
'''

import pandas as pd
import numpy as np
import math as m
from datetime import datetime
import os
from scipy import io

class grabValues():
    def __init__(self,index):
        self.index = index
        self.walldata = []
        loaded = io.loadmat('WallData.mat')
        data = loaded['WallData']
        self.wall_id = [] #initializing id of wall
        names = data[0,0]["UniqueID"] #grabbing an array of the wall id names
        for ii in range(len(names)):
            loc = str(np.concatenate(names[ii]).tolist()) #translate them to dtype string
            loc = loc.replace("[", "") #Removing brackets
            loc = loc.replace("]", "")
            loc = loc.replace("'", "") #removing quotes
            self.wall_id.append(loc) #adding to the wall id list
        
        
        #find wall by checking wall_id[#] where # goes from 0 to 141
        
        
        #walldata.append()
        author = str(data[0,0]["Authors"][self.index,0]) #grabbing authors name
        author = author.replace("[", "") #removing brackets
        author = author.replace("]", "")
        author = author.replace("'", "") #removing quotes
        self.walldata.append(author) #Index 0
        self.walldata.append(self.wall_id[self.index]) #grabbing name of test, Index 1
        
        
        self.walldata.append(float(data[0,0]["Geometry"][0,0]['l'][self.index,0])) #length, Index 2
        self.walldata.append(float(data[0,0]["Geometry"][0,0]['t'][self.index,0])) #thickness, Index 3
        self.walldata.append(float(data[0,0]["Geometry"][0,0]['h'][self.index,0])) #Height of Wall (Physical), Index 4
        self.walldata.append(float(data[0,0]["Geometry"][0,0]['h_measured'][self.index,0])) #Height of wall measured, Index 5
        self.walldata.append(float(data[0,0]["Geometry"][0,0]['h_v'][self.index,0])) #Height of Wall (Effective), Index 6
        self.walldata.append(float(data[0,0]["Geometry"][0,0]['ShearSpan'][self.index,0])) #Shear Span Ratio, Index 7
        self.walldata.append(float(data[0,0]["Geometry"][0,0]['endCover'][self.index,0])) #Coverx (Thickness), Index 8
        self.walldata.append(float(data[0,0]["Geometry"][0,0]['sideCover'][self.index,0])) #Covery (Width), Index 9
        self.walldata.append(float(data[0,0]['Reinf'][0,0]["Confinement"][0,0][self.wall_id[self.index]][0,0]['xdir'][0,0]['b'])) #Boundary Element Length, Index 10

        self.walldata.append(float(data[0,0]['Reinf'][0,0]['rho_be'][index,0]))#reinforcement ratio of vertical bars in boundary, Index 11
       # if 'bars' in data[0,0]['Reinf'][0,0]["Confinement"][0,0][self.wall_id[self.index]][0,0]['ydir']:
       #     barInY = len(data[0,0]['Reinf'][0,0]["Confinement"][0,0][self.wall_id[self.index]][0,0]['ydir'][0,0]['bars'])
       # else:
        barInY = 0
        numOfBars = len(data[0,0]['SectionAnalysis'][0,0][self.wall_id[self.index]][0,0]['Layers'][0,0]['barType'])

        barType = []
        for ii in range(numOfBars):
            loc = str(data[0,0]['SectionAnalysis'][0,0][self.wall_id[self.index]][0,0]['Layers'][0,0]['barType'][ii,0]) #grabbing bar type
            loc = loc.replace("[", "") #removing brackets
            loc = loc.replace("]", "")
            loc = loc.replace("'", "") #removing quotes
            barType.append(loc)

        boundReinLayout = [] #layout of reinforcement layout in boundary
        if barInY == 0:
            boundReinLayout.append(0)
        else:
            for ii in range(numOfBars):
                if barType[ii] == 'b':
                    boundReinLayout.append(str(barInY))
                elif barType[ii] == 'w':
                    break
                    
        self.walldata.append(boundReinLayout) # Index 12
        self.walldata.append(float(data[0,0]['Reinf'][0,0]['d_be'][self.index,0])) #Boundary bar diameter, Index 13
        self.walldata.append(float(data[0,0]['Reinf'][0,0]["Confinement"][0,0][self.wall_id[self.index]][0,0]['xdir'][0,0]['s1'])) #boundary bar spacing, Index 14
        self.walldata.append(float(data[0,0]["Material"][0,0]['fy_be'][self.index,0])) #yield stress of bars in the boundary, Index 15
        self.walldata.append(float(data[0,0]["Material"][0,0]['fu_be'][self.index,0]))#ultimate stress of bars in the boundary, Index 16
        self.walldata.append(float(data[0,0]["Material"][0,0]['fy_be'][self.index,0])/29000)#strain at yield of bars in the boundary, Index 17
        self.walldata.append(float(data[0,0]["Material"][0,0]['eps_ult_be'][self.index,0]))#strain at ultimate of bars in the boundary, Index 18

        self.walldata.append(float(data[0,0]['Reinf'][0,0]['rho_v'][self.index,0]))#reinforcement ratio of vertical bars in web, Index 19

        numOfBars = len(data[0,0]['SectionAnalysis'][0,0][self.wall_id[self.index]][0,0]['Layers'][0,0]['barType'])
        webReinLayout = [] #layout of reinforcement layout in web
        length =numOfBars-2*len(boundReinLayout) 
        for ii in range(length):
            webReinLayout.append(str(2))
        self.walldata.append(webReinLayout) # Index 20
        
        
        self.walldata.append(float(data[0,0]['Reinf'][0,0]['d_v'][self.index,0])) #web bar diameter, Index 21
        
        verBarType = []
        verBarSp = []
        for ii in range(numOfBars):
            loc = str(data[0,0]['SectionAnalysis'][0,0][self.wall_id[self.index]][0,0]['Layers'][0,0]['barType'][ii,0]) #grabbing bar type
            loc = loc.replace("[", "") #removing brackets
            loc = loc.replace("]", "")
            loc = loc.replace("'", "") #removing quotes
            verBarType.append(loc)    
        for ii in range(numOfBars):
            if verBarType[ii] == 'w':
                loc = round(abs((data[0,0]['SectionAnalysis'][0,0][self.wall_id[self.index]][0,0]['Layers'][0,0]['zLoc'][0,0]) - (data[0,0]['SectionAnalysis'][0,0][self.wall_id[self.index]][0,0]['Layers'][0,0]['zLoc'][ii,0])),2)
                verBarSp.append(loc)
        self.walldata.append(verBarSp)# Index 22



        self.walldata.append(float(data[0,0]["Material"][0,0]['fy_v'][self.index,0])) #yield stress of bars in the web, Index 23

        self.walldata.append(float(data[0,0]["Material"][0,0]['fu_v'][self.index,0]))#ultimate stress of bars in the web, Index 24

        self.walldata.append(float(data[0,0]["Material"][0,0]['fy_v'][self.index,0])/29000)#strain at yield of bars in the web, Index 25

        self.walldata.append(float(data[0,0]["Material"][0,0]['eps_ult_v'][self.index,0]))#strain at ultimate of bars in the web, Index 26

        self.walldata.append(float(data[0,0]['Reinf'][0,0]['rho_h'][self.index,0]))#reinforcement ratio of the horizontal bars, Index 27

        self.walldata.append(float(data[0,0]['Reinf'][0,0]['d_h'][self.index,0]))#diameter of horizontal bars, Index 28

        self.walldata.append(float(data[0,0]['Reinf'][0,0]['s_h'][self.index,0]))#spacing of horizontal bars, Index 29

        self.walldata.append(float(data[0,0]["Material"][0,0]['fy_h'][self.index,0]))#yield stress of horizontal bars, Index 30

        self.walldata.append(float(data[0,0]["Material"][0,0]['fu_h'][self.index,0]))#ultimate stress of horizontal bars, Index 31

        self.walldata.append(0.002)#yield strain of horizontal bars, Index 32

        self.walldata.append(float(data[0,0]["Material"][0,0]['eps_ult_h'][self.index,0])) #ultimate strain of horizontal bars, Index 33
        
        self.walldata.append(float(data[0,0]['Reinf'][0,0]['d_hoop'][self.index,0]))#diameter of hoop bars, Index 34

        self.walldata.append(float(data[0,0]['Reinf'][0,0]['s_hoop'][self.index,0]))#spacing for hoop bars, Index 35

        self.walldata.append(float(data[0,0]["Material"][0,0]['fy_hoop'][self.index,0]))#yield strength of hoop bars, Index 36

        self.walldata.append(float(data[0,0]["Material"][0,0]['fu_hoop'][self.index,0]))#ultimate strength of hoop bars, Index 37

        self.walldata.append(0.005)#yield strain of hoop bars, Index 38

        self.walldata.append(float(data[0,0]["Material"][0,0]['eps_ult_hoop'][self.index,0]))#ultimate strain of hoop bars, Index 39

        self.walldata.append(float(data[0,0]["Material"][0,0]['fc'][self.index,0])/1000)#compressive strength of concrete, Index 40

        self.walldata.append(float(data[0,0]["Material"][0,0]['eps'][self.index,0]))#concrete max concrete crushing strain, Index 41

        self.walldata.append((float(4*(1000*self.walldata[40])**(1/2))/1000)) #tension strength of concrete, Index 42

        self.walldata.append(0.005)#density of concrete, Index 43
        
        self.walldata.append(float(data[0,0]["Geometry"][0,0]['BuildingAspectRatio'][self.index,0]))  #aspect ratio of wall, Index 44

        self.walldata.append(float(data[0,0]["Loading"][0,0]['AxialLoad'][self.index,0]))#axial load value, Index 45

        self.walldata.append((data[0,0]["Loading"][0,0]['CyclicHistory'][0,0][self.wall_id[self.index]]))# Index 46
        
        self.walldata.append(float(data[0,0]["ExperimentalData"][0,0][self.wall_id[self.index]][0,0]['dispCap']))#displacement at 80% of maximum force, Index 47
        
 
        